In [ ]:
# coding: utf-8

# # Using SEC EDGAR RESTful data APIs
# 
# This notebook shows how to retrieve information reported by regulated entities to U.S. Securities and Exchange Commision (SEC).
# 
# SEC is maintainig EDGAR system with information about all regulated enties (companies, funds, individuals). Accessing the data is free and there is number of [various ways how to access the data](https://www.sec.gov/os/accessing-edgar-data).
# 
# "data.sec.gov" was created to host RESTful data Application Programming Interfaces (APIs) delivering JSON-formatted data to external customers and to web pages on SEC.gov. These APIs do not require any authentication or API keys to access.
# 
# Currently included in the APIs are the submissions history by filer and the XBRL data from financial statements (forms 10-Q, 10-K,8-K, 20-F, 40-F, 6-K, and their variants).
# 
# The JSON structures are updated throughout the day, in real time, as submissions are disseminated.

# pip install -r requirements.txt

# !jupyter nbextension enable --py widgetsnbextension


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import unicodedata
from bs4 import BeautifulSoup as bs
import requests
from tqdm.notebook import tqdm
import os
import warnings
import boto3
warnings.filterwarnings("ignore")


# ## Finding CIK of company
# 
# EDGAR assigns to filers a unique numerical identifier, known as a Central Index Key (CIK), when they sign up to make filings to the SEC. CIK numbers remain unique to the filer; they are not recycled. 
# 
# List of all CIKs matched with entity name is available for download [(13 MB, text file)](https://www.sec.gov/Archives/edgar/cik-lookup-data.txt). Note that this list includes funds and individuals and is historically cumulative for company names. Thus a given CIK may be associated with multiple names in the case of company or fund name changes, and the list contains some entities that no longer file with the SEC.
# 
# We will be using smaller (611 kB) JSON [kaggle dataset](https://www.kaggle.com/datasets/svendaj/sec-edgar-cik-ticker-exchange), which is sourcing data directly at EDGAR and is input for this notebook. This dataset contains only companies names, CIK, ticker and associated stock exchange.




# Let's convert CIK JSON to pandas DataFrame
# First load the data into python dictionary


CIK_df=pd.read_json("company_tickers.json").T


CIK_df.head()


CIK_df.rename(columns={'cik_str': 'cik', 'title':'name'}, inplace=True)

# ### Finding a particular company based upon the Name they are registered with


# finding companies containing substring in company name
substring = "Black"
CIK_df[CIK_df["name"].str.contains(substring, case=False)]


# # Entity’s current filing history
# 
# Each entity’s current filing history is available at the following URL:
# 
# * https://data.sec.gov/submissions/CIK##########.json
# 
# Where the ########## is the entity’s 10-digit Central Index Key (CIK), including leading zeros.
# 
# This JSON data structure contains metadata such as current name, former name, and stock exchanges and ticker symbols of publicly-traded companies. The object’s property path contains at least one year’s of filing or to 1,000 (whichever is more) of the most recent filings in a compact columnar data array. If the entity has additional filings, files will contain an array of additional JSON files and the date range for the filings each one contains.




# read response from REST API with `requests` library and format it as python dict

import requests
header_full = {
    "User-Agent": "harshit harshit.gola.off@gmail.com",
    "Accept-Encoding": "gzip, deflate",
    "Host": "data.sec.gov"
}



# ## Select the ticker of company used in this example
# 
# Subsequent information retrieval will be using selected `ticker` and associated CIK




# finding company row with given ticker

def get_current_filing_history(url, header):
    company_filings = requests.get(url, headers=header).json()
    company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])
    return company_filings_df
    





BLK_CIK_df = CIK_df[CIK_df['ticker'] == 'BLK']





BLK_CIK_df.head()


# ## Reading from RESTful API
# 
# EDGAR requires that HTTP requests will be identified with proper [UserAgent in header and comply with fair use policy (currently max. 10 requests per second)](https://www.sec.gov/os/accessing-edgar-data). At minimum you need to supply your own e-mail adress in User-Agent field (otherwise you will get 403/Forbiden error). If you will provide Host field, please be sure use data.sec.gov server and not www.sec.gov as mentioned in example (this would result in 404/Not Found error).

# ## Creating DataFrame with submitted filings
# 
# `company_filings["filings"]["recent"]` contains up to 1000 last submitted filings sorted from latest to oldest.










import pandas as pd

def search_dataframe(dataframe, keyword):

    # Create an empty DataFrame to store the matched rows
    matched_rows = pd.DataFrame()
    
    # Iterate over each column in the DataFrame
    for column in dataframe.columns:
        # Convert non-string columns to strings
        if not dataframe[column].dtype == 'object':
            dataframe[column] = dataframe[column].astype(str)
        
        # Use str.contains to search for the keyword in each column
        matched = dataframe[dataframe[column].str.contains(keyword, case=False, na=False)]
        
        # Concatenate the matched rows to the DataFrame
        matched_rows = pd.concat([matched_rows, matched], ignore_index=True)
    
    return matched_rows





keyword = 'BlackRock'

result = search_dataframe(df_history_blk, keyword)
print(result)





df_history.head()


# ## Accessing specific filing document
# 
# Let's download latest Annual Report (10-K). Files are stored in browsable directory structure for CIK and accession-number: 
# * https://www.sec.gov/Archives/edgar/data/{CIK}/{accession-number}/

# Creating a function to create a url and run loop for all the items to download each of the filing htm file




def download_all_forms(df, form, header):
    df_ = df[df.form == form]
    for index, row in tqdm(df_.iterrows(), total=df_.shape[0]):
        url = f"https://www.sec.gov/Archives/edgar/data/{row['cik']}/{row['accessionNumber'].replace('-', '')}/{row['primaryDocument']}"
        req_content = requests.get(url, headers=header).content.decode("utf-8")
        directory = f"data/{row['ticker']}"
        if not os.path.exists(directory):
            os.makedirs(directory)

        with open(f"{directory}/{row['primaryDocument']}", "w") as f:
            f.write(req_content)


# This step is to download all the 10K htm files for the 100 most recent filings into the data folder.




download_all_forms(df_history, '10-K', header)
    





In [2]:


#!/usr/bin/env python
# coding: utf-8

# # 10-K form
# ## Business, Risk, and MD&A
# The function *parse_10k_filing()* parses 10-K forms to extract the following sections: business description, business risk, and management discussioin and analysis. The function takes two arguments, a link and a number indicating the section, and returns a list with the requested sections. Current options are **0(All), 1(Business), 2(Risk), 4(MDA).**
# 
# Caveats:
# The function *parse_10k_filing()* is a parser. You need to feed a SEC text link into it. There are many python and r packages to get a direct link to the fillings.
# 
import re
import unicodedata
import pandas as pd
from bs4 import BeautifulSoup as bs

def parse_10k_filing(file_path, section):
    
    if section not in [0, 1, 2, 3]:
        print("Not a valid section")
        sys.exit()
    
    def get_text(file_path):
        with open(file_path, 'r') as file:
            content = file.read()
        html = bs(content, 'html.parser')
        text = html.get_text()
        text = unicodedata.normalize("NFKD", text).encode('ascii', 'ignore').decode('utf8')
        text = text.split("\n")
        text = " ".join(text)
        return text
    
    def extract_text(text, item_start, item_end):
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return item_text

    text = get_text(file_path)
        
    if section == 1 or section == 0:
        try:
            item1_start = re.compile("item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            item1_end = re.compile("item\s*1a[\.\;\:\-\_]\s*Risk|item\s*2[\.\,\;\:\-\_]\s*Prop", re.IGNORECASE)
            businessText = extract_text(text, item1_start, item1_end)
        except:
            businessText = "Something went wrong!"
        
    if section == 2 or section == 0:
        try:
            item1a_start = re.compile("(?<!,\s)item\s*1a[\.\;\:\-\_]\s*Risk", re.IGNORECASE)
            item1a_end = re.compile("item\s*2[\.\;\:\-\_]\s*Prop|item\s*[1][\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
            riskText = extract_text(text, item1a_start, item1a_end)
        except:
            riskText = "Something went wrong!"
            
    if section == 3 or section == 0:
        try:
            item7_start = re.compile("item\s*[7][\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
            item7_end = re.compile("item\s*7a[\.\;\:\-\_]\sQuanti|item\s*8[\.\,\;\:\-\_]\s*", re.IGNORECASE)
            mdaText = extract_text(text, item7_start, item7_end)
        except:
            mdaText = "Something went wrong!"
    
    if section == 0:
        data = [businessText, riskText, mdaText]
    elif section == 1:
        data = [businessText]
    elif section == 2:
        data = [riskText]
    elif section == 3:
        data = [mdaText]
    
    return data




def parse_all_forms(df, form, header):
    df_ = df[df.form == form]
    df__ = pd.DataFrame()
    for index, row in tqdm(df_.iterrows(), total=df_.shape[0]):
        directory = f"data/{row['ticker']}"
        file_path = f"data/{row['ticker']}/{row['primaryDocument']}"
        section = 0
        
        # Parse the 10-K filing and store the results in a DataFrame
        text_data = parse_10k_filing(file_path, section)
        df_text = pd.DataFrame({'Text': text_data})
        df_text['ticker'] = row['ticker']
        df_text['filepath'] = file_path
        df__ = pd.concat([df_text, df__])

    return df__



df_text = parse_all_forms(df_history , '10-K', header)


## Exporting the resulted dataframe in a csv format.

file_path='./All Reports Data.csv'


#Arranging the index and adding Id column
df_text = df_text.reset_index(drop=True)
df_text['Id'] = df_text.index+1

# Pop the column from its current position
new_column = df_text.pop('Id')

# Insert the column at the front of the DataFrame
df_text.insert(0, 'Id', new_column)

df_text.to_csv(file_path, index=False)



NameError: name 'df_history' is not defined